In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [3]:
df = pd.read_csv('bon_tf_ghi_2020_n.csv')
df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df = df.sort_index()

df = df[df > 0]
df = df.dropna()

df.head()

,0,1,2,3,4,5,6,7,8,9,...,111,112,113,114,115,116,117,118,119,120
time,,,,,,,,,,,,,,,,,,,,,
2020-01-01 13:40:00,0.792169,0.792733,0.792968,0.792157,0.791599,0.791142,0.789827,0.789438,0.789015,0.788927,...,0.792410,0.792439,0.792704,0.793215,0.793409,0.793789,0.793050,0.793040,0.792338,0.792184
2020-01-01 13:45:00,0.792102,0.792444,0.792740,0.792217,0.791788,0.790531,0.789976,0.789481,0.789733,0.790200,...,0.792126,0.791741,0.791753,0.791973,0.792708,0.793137,0.793288,0.793451,0.793107,0.793048
2020-01-01 13:50:00,0.793457,0.794285,0.793274,0.792211,0.791347,0.790630,0.790719,0.790296,0.790957,0.791019,...,0.792357,0.792143,0.792446,0.792770,0.794127,0.793923,0.794680,0.794090,0.793897,0.794338
2020-01-01 13:55:00,0.795044,0.795560,0.795075,0.794183,0.793174,0.792442,0.792130,0.791738,0.792113,0.792873,...,0.794089,0.794074,0.794943,0.795199,0.796139,0.796155,0.796623,0.796170,0.796341,0.796915
2020-01-01 14:00:00,0.797914,0.797860,0.798208,0.797192,0.796637,0.796254,0.795041,0.795250,0.795430,0.795695,...,0.796282,0.796791,0.797493,0.798360,0.799728,0.800105,0.800167,0.799595,0.799547,0.799892


In [5]:
df_ghi = pd.read_csv('\bon_ghics_2020.csv')
df_ghi['time'] = pd.to_datetime(df_ghi['time'])
df_ghi = df_ghi.set_index('time')
df_ghi = df_ghi[df_ghi['ghi'] > 0]

df_ghi['CSI'] = df_ghi['ghi']/df_ghi['ghics_nsrdb'] # forecasting on clear-sky index, CSI
df_ghi.head()

,ghics_pvlib,ghics_tl,ghics_tla,ghi,zen,dw_solar,GHI,ghics_rest,ghics_mc,CSI
time,,,,,,,,,,
2020-01-01 15:50:00,328.551397,327.419917,333.794446,347.92,69.61,347.92,359.0,359.0,363.2448,0.969136
2020-01-01 15:55:00,338.063247,336.924446,343.339106,357.34,69.12,357.34,369.0,369.0,373.0872,0.968401
2020-01-01 16:00:00,347.243301,346.097706,352.549600,365.98,68.66,365.98,378.0,378.0,382.5876,0.968201
2020-01-01 16:05:00,356.084862,354.932962,361.419405,374.50,68.20,374.50,387.0,387.0,391.7388,0.967700
2020-01-01 16:10:00,364.581653,363.423902,369.942400,383.04,67.77,383.04,395.0,395.0,400.5348,0.969722


In [7]:
train_months = [1,2,4,5,7,8,10,11]
#valid_months = []
test_months = [3,6,9,12]
train_index = [index for index in df_ghi.index if index.month in train_months]
test_index = [index for index in df_ghi.index if index.month in test_months]
len(train_index),len(test_index),len(train_index)+len(test_index),df_ghi.shape[0]

(17220, 5479, 22699, 22699)

In [8]:
tr_index = train_index 
te_index = test_index 
len(tr_index),len(te_index)

(17220, 5479)

In [9]:
# select the time indexes
index_ = []
c_index = df.index
all_index = tr_index + te_index
for index in all_index:
    if index in c_index and (index - datetime.timedelta(minutes=5) in c_index 
        and index - datetime.timedelta(minutes=10) in c_index
        and index - datetime.timedelta(minutes=15) in c_index
        and index - datetime.timedelta(minutes=20) in c_index 
        and index - datetime.timedelta(minutes=25) in c_index 
        and index - datetime.timedelta(minutes=30) in c_index
        and index - datetime.timedelta(minutes=35) in c_index 
        and index - datetime.timedelta(minutes=40) in c_index
        and index - datetime.timedelta(minutes=45) in c_index 
        and index - datetime.timedelta(minutes=50) in c_index
        and index - datetime.timedelta(minutes=55) in c_index 
        and index - datetime.timedelta(minutes=60) in c_index
        and index + datetime.timedelta(minutes=15) in all_index 
        and index + datetime.timedelta(minutes=30) in all_index
        and index + datetime.timedelta(minutes=45) in all_index 
        and index + datetime.timedelta(minutes=60) in all_index
        and index + datetime.timedelta(minutes=90) in all_index 
        and index + datetime.timedelta(minutes=120) in all_index
        and index + datetime.timedelta(minutes=150) in all_index 
        and index + datetime.timedelta(minutes=180) in all_index
                            ):
        index_.append(index)
len(index_)

5104

In [10]:
tr_index_00 = [index for index in index_ if index in tr_index]

tr_index_60 = [index - datetime.timedelta(minutes=60) for index in tr_index_00]
tr_index_55 = [index - datetime.timedelta(minutes=55) for index in tr_index_00]
tr_index_50 = [index - datetime.timedelta(minutes=50) for index in tr_index_00]
tr_index_45 = [index - datetime.timedelta(minutes=45) for index in tr_index_00]
tr_index_40 = [index - datetime.timedelta(minutes=40) for index in tr_index_00]
tr_index_35 = [index - datetime.timedelta(minutes=35) for index in tr_index_00]
tr_index_30 = [index - datetime.timedelta(minutes=30) for index in tr_index_00] 
tr_index_25 = [index - datetime.timedelta(minutes=25) for index in tr_index_00]
tr_index_20 = [index - datetime.timedelta(minutes=20) for index in tr_index_00]
tr_index_15 = [index - datetime.timedelta(minutes=15) for index in tr_index_00]
tr_index_10 = [index - datetime.timedelta(minutes=10) for index in tr_index_00]
tr_index_05 = [index - datetime.timedelta(minutes=5) for index in tr_index_00]

tr_index15  = [index + datetime.timedelta(minutes=15) for index in tr_index_00]
tr_index30  = [index + datetime.timedelta(minutes=30) for index in tr_index_00]
tr_index45  = [index + datetime.timedelta(minutes=45) for index in tr_index_00]
tr_index60  = [index + datetime.timedelta(minutes=60) for index in tr_index_00]
tr_index90  = [index + datetime.timedelta(minutes=90) for index in tr_index_00]
tr_index120  = [index + datetime.timedelta(minutes=120) for index in tr_index_00]
tr_index150  = [index + datetime.timedelta(minutes=150) for index in tr_index_00]
tr_index180  = [index + datetime.timedelta(minutes=180) for index in tr_index_00]

In [11]:
times = ['_60','_55','_50','_45','_40','_35','_30','_25','_20','_15','_10','_05','_00']
indexes = [tr_index_60,tr_index_55,tr_index_50,tr_index_45,tr_index_40,tr_index_35,tr_index_30,
           tr_index_25,tr_index_20,tr_index_15,tr_index_10,tr_index_15,tr_index_00]

inputs = {}

for time,index in zip(times,indexes):
    inputs['train'+time] = df.loc[index].values.reshape(len(index),11,11)

#inputs

In [13]:
te_index_00 = [index for index in index_ if index in te_index]

te_index_60 = [index - datetime.timedelta(minutes=60) for index in te_index_00]
te_index_55 = [index - datetime.timedelta(minutes=55) for index in te_index_00]
te_index_50 = [index - datetime.timedelta(minutes=50) for index in te_index_00]
te_index_45 = [index - datetime.timedelta(minutes=45) for index in te_index_00]
te_index_40 = [index - datetime.timedelta(minutes=40) for index in te_index_00]
te_index_35 = [index - datetime.timedelta(minutes=35) for index in te_index_00]
te_index_30 = [index - datetime.timedelta(minutes=30) for index in te_index_00] 
te_index_25 = [index - datetime.timedelta(minutes=25) for index in te_index_00]
te_index_20 = [index - datetime.timedelta(minutes=20) for index in te_index_00]
te_index_15 = [index - datetime.timedelta(minutes=15) for index in te_index_00]
te_index_10 = [index - datetime.timedelta(minutes=10) for index in te_index_00]
te_index_05 = [index - datetime.timedelta(minutes=5) for index in te_index_00]

te_index15  = [index + datetime.timedelta(minutes=15) for index in te_index_00]
te_index30  = [index + datetime.timedelta(minutes=30) for index in te_index_00]
te_index45  = [index + datetime.timedelta(minutes=45) for index in te_index_00]
te_index60  = [index + datetime.timedelta(minutes=60) for index in te_index_00]
te_index90  = [index + datetime.timedelta(minutes=90) for index in te_index_00]
te_index120  = [index + datetime.timedelta(minutes=120) for index in te_index_00]
te_index150  = [index + datetime.timedelta(minutes=150) for index in te_index_00]
te_index180  = [index + datetime.timedelta(minutes=180) for index in te_index_00]

In [14]:
times = ['_60','_55','_50','_45','_40','_35','_30','_25','_20','_15','_10','_05','_00']
indexes = [te_index_60,te_index_55,te_index_50,te_index_45,te_index_40,te_index_35,te_index_30,
           te_index_25,te_index_20,te_index_15,te_index_10,te_index_05,te_index_00]

te_inputs = {}

for time,index in zip(times,indexes):
    te_inputs['test'+time] = df.loc[index].values.reshape(len(index),11,11)

#te_inputs

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Attention, Dense, Layer, Input, Conv2D, MaxPool2D, concatenate, Flatten, Dropout, BatchNormalization
from tensorflow.keras.utils import plot_model
from tensorflow.keras import regularizers, callbacks
import tensorflow.keras.backend as K 

In [16]:
class attention(Layer):
    
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)
        
    def build(self,input_shape):
        self.W = self.add_weight(name = 'attention_weight', shape = (input_shape[-1],1),
                                 initializer = 'random_normal', trainable = True)
        self.b = self.add_weight(name = 'attention_bias', shape = (input_shape[1],1),
                                 initializer = 'zeros', trainable = True)
        super(attention,self).build(input_shape)
        
    def call(self,x):
        e = K.tanh(K.dot(x,self.W)+self.b)
        e = K.squeeze(e,axis=-1)
        alpha = K.softmax(e)
        alpha = K.expand_dims(alpha,axis=-1)
        context = x*alpha
        context = K.sum(context,axis=1)
        return context

In [17]:
def input_one(hp,input_x):
    
    x = BatchNormalization()(input_x)
    
    return x

def input_all(hp,input_60,input_55,input_50,input_45,input_40,input_35,input_30,input_25,input_20,input_15,input_10,input_05,input_00):
    
    x_60 = input_one(hp,input_60)
    x_55 = input_one(hp,input_55)
    x_50 = input_one(hp,input_50)
    x_45 = input_one(hp,input_45)
    x_40 = input_one(hp,input_40)
    x_35 = input_one(hp,input_35)
    x_30 = input_one(hp,input_30)
    x_25 = input_one(hp,input_25)
    x_20 = input_one(hp,input_20)
    x_15 = input_one(hp,input_15)
    x_10 = input_one(hp,input_10)
    x_05 = input_one(hp,input_05)
    x_00 = input_one(hp,input_00)
    
    x = concatenate([x_60,x_55,x_50,x_45,x_40,x_35,x_30,x_25,x_20,x_15,x_10,x_05,x_00])
    
    return x

In [18]:
def model_builder(hp):
    
    input_60 = Input(shape=[11,11,1])
    input_55 = Input(shape=[11,11,1])
    input_50 = Input(shape=[11,11,1])
    input_45 = Input(shape=[11,11,1])
    input_40 = Input(shape=[11,11,1])
    input_35 = Input(shape=[11,11,1])
    input_30 = Input(shape=[11,11,1])
    input_25 = Input(shape=[11,11,1])
    input_20 = Input(shape=[11,11,1])
    input_15 = Input(shape=[11,11,1])
    input_10 = Input(shape=[11,11,1])
    input_05 = Input(shape=[11,11,1])
    input_00 = Input(shape=[11,11,1])

    x = input_all(hp,input_60,input_55,input_50,input_45,input_40,input_35,input_30,input_25,input_20,input_15,input_10,input_05,input_00)

    hp_layer_1 = hp.Int('layer_1', min_value=16, max_value=128, step=16)
    hp_layer_2 = hp.Int('layer_2', min_value=8, max_value=64, step=8)
    hp_layer_3 = hp.Int('layer_3', min_value=8, max_value=64, step=8)
    hp_layer_4 = hp.Int('layer_4', min_value=8, max_value=64, step=8)
    
    hp_activation_1 = hp.Choice('activation_1', values=['relu','gelu','selu'])
    hp_activation_2 = hp.Choice('activation_2', values=['relu','gelu','selu'])
    hp_activation_3 = hp.Choice('activation_3', values=['relu','gelu','selu'])
    hp_activation_4 = hp.Choice('activation_4', values=['relu','gelu','selu','linear'])
    
    x = Conv2D(filters=hp_layer_1, kernel_size = 3, activation = hp_activation_1)(x)
    x = BatchNormalization()(x)
    x = MaxPool2D((2,2))(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=hp_layer_2, kernel_size = 3, activation = hp_activation_2)(x)
    x = BatchNormalization()(x)
    # x = MaxPool2D((2,2))(x)
    # x = BatchNormalization()(x)
    x = attention()(x)
    x = BatchNormalization()(x)

    x = Flatten()(x)

    x = Dense(units=hp_layer_3, activation = hp_activation_3, kernel_regularizer = regularizers.l2(0.01))(x)

    x = Dense(units=hp_layer_4, activation = hp_activation_4)(x)

    output_15 = Dense(1, activation = 'relu')(x)

    output_30 = Dense(1, activation = 'relu')(x)

    output_45 = Dense(1, activation = 'relu')(x)

    output_60 = Dense(1, activation = 'relu')(x)
    
    output_90 = Dense(1, activation = 'relu')(x)

    output_120 = Dense(1, activation = 'relu')(x)

    output_150 = Dense(1, activation = 'relu')(x)

    output_180 = Dense(1, activation = 'relu')(x)

    model = Model(inputs = [input_60,input_55,input_50,input_45,input_40,
                            input_35,input_30,input_25,input_20,input_15,
                            input_10,input_05,input_00],
                  outputs = [output_15,output_30,output_45,output_60,output_90,output_120,output_150,output_180]
                 )
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-5,1e-4,2e-4,5e-4,1e-3,2e-3,1e-2])
    
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
             loss = 'huber',
             metrics = tf.keras.metrics.MeanAbsoluteError())

    return model

In [19]:
import keras_tuner as kt

tuner = kt.BayesianOptimization(model_builder,
                                objective=kt.Objective("val_loss",direction="min"),
                                max_trials=50,
                                seed=3,
                                directory='forecasting',
                                project_name='bon3h_sat-dl')

In [20]:
early_stop = callbacks.EarlyStopping(monitor='val_loss',mode='min',patience=5)
callback_list = [early_stop]

In [21]:
train_output_15 = df_ghi.loc[tr_index15,'CSI'].values.reshape(len(tr_index15),1)
train_output_30 = df_ghi.loc[tr_index30,'CSI'].values.reshape(len(tr_index30),1)
train_output_45 = df_ghi.loc[tr_index45,'CSI'].values.reshape(len(tr_index45),1)
train_output_60 = df_ghi.loc[tr_index60,'CSI'].values.reshape(len(tr_index60),1)
train_output_90 = df_ghi.loc[tr_index90,'CSI'].values.reshape(len(tr_index90),1)
train_output_120 = df_ghi.loc[tr_index120,'CSI'].values.reshape(len(tr_index120),1)
train_output_150 = df_ghi.loc[tr_index150,'CSI'].values.reshape(len(tr_index150),1)
train_output_180 = df_ghi.loc[tr_index180,'CSI'].values.reshape(len(tr_index180),1)

test_output_15 = df_ghi.loc[te_index15,'CSI'].values.reshape(len(te_index15),1)
test_output_30 = df_ghi.loc[te_index30,'CSI'].values.reshape(len(te_index30),1)
test_output_45 = df_ghi.loc[te_index45,'CSI'].values.reshape(len(te_index45),1)
test_output_60 = df_ghi.loc[te_index60,'CSI'].values.reshape(len(te_index60),1)
test_output_90 = df_ghi.loc[te_index90,'CSI'].values.reshape(len(te_index90),1)
test_output_120 = df_ghi.loc[te_index120,'CSI'].values.reshape(len(te_index120),1)
test_output_150 = df_ghi.loc[te_index150,'CSI'].values.reshape(len(te_index150),1)
test_output_180 = df_ghi.loc[te_index180,'CSI'].values.reshape(len(te_index180),1)

In [22]:
tuner.search([inputs[key] for key in inputs.keys()],
             [train_output_15,train_output_30,train_output_45,train_output_60,
              train_output_90,train_output_120,train_output_150,train_output_180], 
             epochs=5, 
             validation_split=0.2, 
             shuffle = True,
             callbacks=callback_list)

Trial 50 Complete [00h 00m 23s]
val_output_15_mean_absolute_error: 0.08690578490495682

Best val_output_15_mean_absolute_error So Far: 0.035396188497543335
Total elapsed time: 00h 17m 39s
INFO:tensorflow:Oracle triggered exit


In [24]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
history = model.fit([[inputs[key] for key in inputs.keys()]],
                    [train_output_15,train_output_30,train_output_45,train_output_60,
                     train_output_90,train_output_120,train_output_150,train_output_180],
                    epochs=200, validation_split=0.2, shuffle = True,
                    callbacks=callback_list)

Epoch 1/100
88/88 [==============================] - 6s 43ms/step - loss: 1.1345 - output_15_loss: 0.1197 - dense_8_loss: 0.0683 - dense_9_loss: 0.0781 - output_60_loss: 0.0780 - output_90_loss: 0.0481 - dense_10_loss: 0.0971 - dense_11_loss: 0.1511 - output_180_loss: 0.2918 - output_15_mean_absolute_error: 0.3416 - dense_8_mean_absolute_error: 0.2515 - dense_9_mean_absolute_error: 0.2648 - output_60_mean_absolute_error: 0.2690 - output_90_mean_absolute_error: 0.2057 - dense_10_mean_absolute_error: 0.3038 - dense_11_mean_absolute_error: 0.4086 - output_180_mean_absolute_error: 0.6554 - val_loss: 0.7669 - val_output_15_loss: 0.0295 - val_dense_8_loss: 0.0034 - val_dense_9_loss: 0.0208 - val_output_60_loss: 0.0038 - val_output_90_loss: 0.0325 - val_dense_10_loss: 0.0683 - val_dense_11_loss: 0.0045 - val_output_180_loss: 0.4216 - val_output_15_mean_absolute_error: 0.2346 - val_dense_8_mean_absolute_error: 0.0405 - val_dense_9_mean_absolute_error: 0.1956 - val_output_60_mean_absolute_error

In [82]:
model.save('saved_model/bon_sat-dl_3h')

INFO:tensorflow:Assets written to: saved_model/bon_dl_3h\assets


In [25]:
model.evaluate(x = [te_inputs[key] for key in te_inputs.keys()],
               y = [test_output_15,test_output_30,test_output_45,test_output_60,
                    test_output_90,test_output_120,test_output_150,test_output_180]
              )

50/50 [==============================] - 2s 22ms/step - loss: 0.0404 - output_15_loss: 0.0044 - dense_8_loss: 0.0040 - dense_9_loss: 0.0040 - output_60_loss: 0.0038 - output_90_loss: 0.0043 - dense_10_loss: 0.0054 - dense_11_loss: 0.0056 - output_180_loss: 0.0068 - output_15_mean_absolute_error: 0.0615 - dense_8_mean_absolute_error: 0.0609 - dense_9_mean_absolute_error: 0.0540 - output_60_mean_absolute_error: 0.0504 - output_90_mean_absolute_error: 0.0546 - dense_10_mean_absolute_error: 0.0707 - dense_11_mean_absolute_error: 0.0692 - output_180_mean_absolute_error: 0.0762


[0.040424514561891556,
 0.004374384880065918,
 0.003995513077825308,
 0.00397964334115386,
 0.0038480593357235193,
 0.004328198730945587,
 0.0054413252510130405,
 0.005638363771140575,
 0.006826439872384071,
 0.06152596324682236,
 0.060928937047719955,
 0.05404972285032272,
 0.050391871482133865,
 0.05457311496138573,
 0.07072149217128754,
 0.06924276053905487,
 0.07619927823543549]

In [27]:
y_predict = model.predict(x = [te_inputs[key] for key in te_inputs.keys()])

In [52]:
from sklearn import metrics
def rmse_fs(y,test_index):
    ns_csi_00 = df_ghi.loc[te_index_00,'CSI'].values.reshape(len(te_index_00),1)
    ns_clear = df_ghi.loc[test_index,'ghics_rest'].values.reshape(len(test_index),1)
    test_output = df_ghi.loc[test_index,'CSI'].values.reshape(len(test_index),1)
    rmse1 = np.sqrt(metrics.mean_squared_error(y*ns_clear,test_output*ns_clear))
    nrmse1 = rmse1/np.mean(test_output*ns_clear)
    rmse0 = np.sqrt(metrics.mean_squared_error(ns_csi_00*ns_clear,test_output*ns_clear))
    nrmse0 = rmse0/np.mean(test_output*ns_clear)
    mbe1 = np.mean([x-y for x,y in zip(y*ns_clear,test_output*ns_clear)])
    nmbe1 = mbe1/np.mean(test_output*ns_clear)
    mbe0 = np.mean([x-y for x,y in zip(ns_csi_00*ns_clear,test_output*ns_clear)])
    nmbe0 = mbe0/np.mean(test_output*ns_clear)
    fs = 1-rmse1/rmse0
    return nrmse1,nrmse0,nmbe1,nmbe0,fs

In [53]:
tests = [te_index15,te_index30,te_index45,te_index60,te_index90,te_index120,te_index150,te_index180]
for x,y in zip([y_predict[i] for i in range(8)],tests):
    print(rmse_fs(x,y))

(0.09207961759708261, 0.0943048664610409, 0.045719306277421455, -0.0017308516233758577, 0.023596331212425548)
(0.08881981125910418, 0.10322693889593071, 0.042686609219969696, -0.003434878447254541, 0.1395675178487199)
(0.09017659846625717, 0.11110217813365518, 0.0273211228502112, -0.0045245640688892814, 0.18834535937022479)
(0.09006663018237414, 0.11275918195468787, 0.017847306058294897, -0.005058630691549471, 0.2012479283632327)
(0.09604606671924919, 0.1238699408667341, 0.027393167219685117, -0.005347537709640315, 0.22462167942276912)
(0.10485529151705614, 0.13456515288358112, 0.03709153660312905, -0.0033561988355503445, 0.2207842129249349)
(0.10443853840184103, 0.14080582724103732, 0.014157018113421299, 0.001953408683544789, 0.25827971435401775)
(0.1108757531171578, 0.14965130424717635, 0.022620281348437632, 0.002502101724075544, 0.2591060019495298)
